In [4]:
# Add the combined dataset, can do in research time 6-8 next week. Once that is done, apply a simple transformer architecture (maybe synthesis control?), and next week is analyzing results. Then, week 5 you can start writing the paper. 

In [52]:
# Everything done, analyze the results for Gradient Boosting, LSTMs, and Transformers and maybe combine the results to form an ensemble model. 

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
import seaborn as sns
import matplotlib.pyplot as plt
import copy

In [2]:
data = pd.read_csv("Season_averages_NBA_1960-2023.csv")
del data['Unnamed: 0']

In [172]:
player_season_counts = data.groupby(['Player', 'season']).count()
player_season_counts[player_season_counts['games_played'] > 1]

games_played  player_id  min  fgm  fga  fg3m  fg3a  \
Player          season                                                       
Asante Gist     2021               2          2    2    2    2     2     2   
Charles Jones   1984               2          2    2    2    2     2     2   
                1985               2          2    2    2    2     2     2   
                1987               2          2    2    2    2     2     2   
                1988               2          2    2    2    2     2     2   
Charles Smith   1989               2          2    2    2    2     2     2   
                1990               2          2    2    2    2     2     2   
                1995               2          2    2    2    2     2     2   
Chris Johnson   2012               2          2    2    2    2     2     2   
Dru Smith       2021               2          2    2    2    2     2     2   
Eddie Johnson   1981               2          2    2    2    2     2     2   
                1982               2          2    2    2    2     2     2   
                1983               2          2    2    2    2     2     2   
                1984               2          2    2    2    2     2     2   
                1985               2          2    2    2    2     2     2   
                1986               2          2    2    2    2     2     2   
George Johnson  1972               2          2    2    2    2     2     2   
                1973               2          2    2    2    2     2     2   
                1978               2          2    2    2    2     2     2   
                1979               2          2    2    2    2     2     2   
                1980               2          2    2    2    2     2     2   
                1981               2          2    2    2    2     2     2   
                1982               2          2    2    2    2     2     2   
                1984               2          2    2    2    2     2     2   
                1985               2          2    2    2    2     2     2   
JT Thor         2021               2          2    2    2    2     2     2   
Jaime Echenique 2021               2          2    2    2    2     2     2   
Jamorko Pickett 2021               2          2    2    2    2     2     2   
Jordan Goodwin  2021               2          2    2    2    2     2     2   
Justin Jackson  2021               2          2    2    2    2     2     2   
Justin Robinson 2021               2          2    2    2    2     2     2   
Malcolm Hill    2021               2          2    2    2    2     2     2   
Marcus Williams 2007               2          2    2    2    2     2     2   
                2008               2          2    2    2    2     2     2   
Micah Potter    2021               2          2    2    2    2     2     2   
Michael Smith   1994               2          2    2    2    2     2     2   
Olivier Sarr    2021               2          2    2    2    2     2     2   
Tony Mitchell   2013               2          2    2    2    2     2     2   

                        ftm  fta  oreb  dreb  reb  ast  stl  blk  turnover  \
Player          season                                                       
Asante Gist     2021      2    2     2     2    2    2    2    2         2   
Charles Jones   1984      2    2     2     2    2    2    2    2         2   
                1985      2    2     2     2    2    2    2    2         2   
                1987      2    2     2     2    2    2    2    2         2   
                1988      2    2     2     2    2    2    2    2         2   
Charles Smith   1989      2    2     2     2    2    2    2    2         2   
                1990      2    2     2     2    2    2    2    2         2   
                1995      2    2     2     2    2    2    2    2         2   
Chris Johnson   2012      2    2     2     2    2    2    2    2         2   
Dru Smith       2021      2    2     2     2 

In [3]:
data['Player'] = data['first_name'] + ' ' + data['last_name']

In [4]:
data['Rookie'] = 0

In [9]:
data.columns

Index(['games_played', 'player_id', 'season', 'min', 'fgm', 'fga', 'fg3m',
       'fg3a', 'ftm', 'fta', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk',
       'turnover', 'pf', 'pts', 'fg_pct', 'fg3_pct', 'ft_pct', 'first_name',
       'last_name', 'position', 'height_feet', 'height_inches',
       'weight_pounds', 'Player', 'Rookie'],
      dtype='object')

In [11]:
data['min'] = data['min'].str.replace(':','.').astype('float')

In [13]:
data

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct,first_name,last_name,position,height_feet,height_inches,weight_pounds,Player,Rookie
0,12,46397639,1960,20.45,3.333,8.333,0.000,0.000,0.750,1.167,0.000,0.000,6.750,0.417,0.000,0.000,0.000,2.167,7.417,0.400,0.000,0.643,Mike,Farmer,NaN,NaN,NaN,NaN,Mike Farmer,0
1,39,46396616,1960,24.01,3.000,8.205,0.000,0.000,1.718,2.615,0.000,0.000,4.718,1.000,0.000,0.000,0.000,2.769,7.718,0.366,0.000,0.657,Si,Green,NaN,NaN,NaN,NaN,Si Green,0
2,18,46398147,1960,25.13,5.222,12.167,0.000,0.000,3.611,4.167,0.000,0.000,4.667,1.389,0.000,0.000,0.000,2.222,14.056,0.429,0.000,0.867,Dave,Gambee,NaN,NaN,NaN,NaN,Dave Gambee,0
3,39,46392456,1960,31.09,6.564,17.359,0.000,0.000,4.282,5.385,0.000,0.000,2.846,5.359,0.000,0.000,0.000,2.615,17.436,0.378,0.000,0.795,Bob,Cousy,NaN,NaN,NaN,NaN,Bob Cousy,0
4,33,46392455,1960,18.58,3.333,7.697,0.000,0.000,2.697,3.242,0.000,0.000,6.030,0.182,0.000,0.000,0.000,2.818,9.364,0.433,0.000,0.832,Larry,Foust,NaN,NaN,NaN,NaN,Larry Foust,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23067,18,56677832,2023,9.43,1.556,3.667,0.667,1.778,0.333,0.389,0.444,1.111,1.556,1.111,0.500,0.333,0.556,0.944,4.111,0.424,0.375,0.857,Jarace,Walker,F,NaN,NaN,NaN,Jarace Walker,0
23068,53,56677844,2023,26.50,2.981,6.698,1.000,2.736,1.377,1.962,0.887,3.189,4.075,1.755,0.849,0.774,1.491,2.170,8.340,0.445,0.366,0.702,Bilal,Coulibaly,G,NaN,NaN,NaN,Bilal Coulibaly,0
23069,14,56913527,2023,9.47,2.071,4.214,0.286,1.000,2.357,3.000,0.714,1.357,2.071,0.286,0.214,0.000,0.143,0.571,6.786,0.492,0.286,0.786,Ricky,Council IV,G,NaN,NaN,NaN,Ricky Council IV,0
23070,7,56677843,2023,9.51,1.571,2.571,0.143,0.571,0.857,0.857,0.857,1.000,1.857,0.857,0.000,0.143,0.429,0.714,4.143,0.611,0.250,1.000,Noah,Clowney,F,NaN,NaN,NaN,Noah Clowney,0


In [15]:
data_rookie = pd.read_csv("college_stats.csv")

In [17]:
data_rookie_last = data_rookie.groupby('Player').last().reset_index()

In [19]:
draft = pd.read_csv("Draft_picks_data.csv")

In [21]:
data_rookie = data.groupby('player_id').first().reset_index()[['player_id', 'Player']]

In [23]:
merged_data = pd.merge(data_rookie, data_rookie_last, on='Player')

In [25]:
merged_data = pd.merge(merged_data, draft, on='Player')

In [27]:
merged_data.head()

,player_id,Player,Season,School,Conf,Class,G,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,Unnamed: 29,SOS,Team,Affiliation,Year,Round Number,Round Pick,Overall Pick
0,3,Steven Adams,2012-13,Pitt,Big East,FR,32.0,32.0,23.4,3.1,5.5,0.571,3.1,5.5,0.571,0.0,0.0,NaN,1.0,2.2,0.443,2.8,3.5,6.3,0.6,0.7,2.0,1.1,1.6,7.2,NaN,5.66,Oklahoma City Thunder,Pittsburgh,2013,1,12,12
1,4,Bam Adebayo,2016-17,Kentucky,SEC,FR,38.0,38.0,30.1,4.5,7.5,0.599,4.5,7.5,0.599,0.0,0.0,NaN,4.1,6.2,0.653,3.1,4.9,8.0,0.8,0.7,1.5,1.7,2.6,13.0,NaN,10.49,Miami Heat,Kentucky,2017,1,14,14
2,6,LaMarcus Aldridge,2005-06,Texas,Big 12,SO,37.0,37.0,33.7,5.9,10.4,0.569,5.9,10.4,0.569,0.0,0.0,NaN,3.2,4.9,0.646,3.6,5.5,9.2,0.5,1.4,2.0,1.6,2.5,15.0,NaN,6.99,Chicago Bulls,Texas,2006,1,2,2
3,8,Grayson Allen,2017-18,Duke,ACC,SR,37.0,37.0,35.6,4.9,11.7,0.418,2.1,4.2,0.503,2.8,7.5,0.37,2.9,3.4,0.850,0.5,2.8,3.3,4.6,1.7,0.1,2.1,2.1,15.5,NaN,9.70,Utah Jazz,Duke,2018,1,21,21
4,9,Jarrett Allen,2016-17,Texas,Big 12,FR,33.0,33.0,32.2,5.4,9.6,0.566,5.4,9.4,0.579,0.0,0.2,0.00,2.5,4.5,0.564,3.0,5.4,8.4,0.8,0.6,1.5,2.5,2.1,13.4,NaN,12.75,Brooklyn Nets,Texas,2017,1,22,22


In [29]:
merged_data['3P%'].fillna(0, inplace = True)

/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_63171/418591076.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['3P%'].fillna(0, inplace = True)


In [31]:
del merged_data['Unnamed: 29']
del merged_data['ORB']
del merged_data['DRB']
merged_data['GS'].fillna(merged_data['G'], inplace = True)

/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_63171/4052666328.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['GS'].fillna(merged_data['G'], inplace = True)


In [33]:
merged_data.rename(columns = {'G':'games_played', 'MP':'min', 'FG':'fgm', 'FGA':'fga', 'FG%':'fg_pct', '3P':'fg3m', '3PA':'fg3a', '3P%':'fg3_pct', 'FT':'ftm', 'FTA':'fta', 'FT%':'ft_pct', 'TRB':'reb', 'AST':'ast', 'STL':'stl', 'BLK':'blk', 'TOV':'turnover', 'PF':'pf', 'PTS':'pts', 'Season':'season'}, inplace = True)

In [35]:
merged_data['season'] = (
    merged_data['season']        
        .astype(str)           
        .str.split('-', n=1)      
        .str[0]                   
        .astype(int)              
)

In [37]:
merged_data['Rookie'] = 1

In [39]:
data_final = pd.concat([data, merged_data])

In [41]:
data_final.drop(['School', 'Conf', 'Class', 'GS', '2P', '2PA', '2P%', 'SOS', 'Team', 'Affiliation',	'Year', 'Round Number', 'Round Pick', 'Overall Pick'], axis = 1, inplace = True)

In [43]:
data_final.rename(columns = {'season':'Year'}, inplace = True)

In [45]:
data.sort_values(by='season', ascending=True)

,games_played,player_id,season,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct,first_name,last_name,position,height_feet,height_inches,weight_pounds,Player,Rookie
0,12,46397639,1960,20.45,3.333,8.333,0.000,0.000,0.750,1.167,0.000,0.000,6.750,0.417,0.000,0.000,0.000,2.167,7.417,0.400,0.000,0.643,Mike,Farmer,NaN,NaN,NaN,NaN,Mike Farmer,0
66,7,46395585,1960,21.34,3.143,5.000,0.000,0.000,5.429,5.857,0.000,0.000,2.429,1.000,0.000,0.000,0.000,3.143,11.714,0.629,0.000,0.927,Ken,Sears,NaN,NaN,NaN,NaN,Ken Sears,0
65,20,46391393,1960,36.33,7.550,18.450,0.000,0.000,7.950,9.400,0.000,0.000,9.400,3.050,0.000,0.000,0.000,2.550,23.050,0.409,0.000,0.846,Dolph,Schayes,NaN,NaN,NaN,NaN,Dolph Schayes,0
64,33,46398668,1960,18.32,2.515,6.394,0.000,0.000,0.939,1.364,0.000,0.000,4.333,0.636,0.000,0.000,0.000,1.939,5.970,0.393,0.000,0.689,Thomas,Sanders,NaN,NaN,NaN,NaN,Thomas Sanders,0
63,6,46398161,1960,11.10,3.667,9.167,0.000,0.000,0.167,0.333,0.000,0.000,3.333,0.667,0.000,0.000,0.000,2.000,7.500,0.400,0.000,0.500,Joe,Ruklick,NaN,NaN,NaN,NaN,Joe Ruklick,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21566,30,457,2023,20.02,1.967,3.833,0.267,0.900,0.800,1.200,1.367,3.367,4.733,1.233,1.233,0.233,0.933,1.467,5.000,0.513,0.296,0.667,Jarred,Vanderbilt,F,6.0,9.0,214.0,Jarred Vanderbilt,0
20478,54,38,2023,29.33,4.111,8.778,2.889,6.481,0.296,0.463,0.667,3.278,3.944,1.352,0.815,0.111,0.704,2.056,11.407,0.468,0.446,0.640,Malik,Beasley,G,6.0,5.0,195.0,Malik Beasley,0
17893,50,214,2023,33.21,4.980,10.520,2.060,4.680,1.100,1.320,1.340,4.520,5.860,4.780,0.800,0.720,1.860,1.660,13.120,0.473,0.440,0.833,Jrue,Holiday,G,6.0,4.0,205.0,Jrue Holiday,0
22617,3,17896041,2023,15.40,2.000,5.333,1.333,2.667,2.667,2.667,0.667,0.667,1.333,1.000,1.000,0.333,1.000,1.000,8.000,0.375,0.500,1.000,Keon,Johnson,G,NaN,NaN,NaN,Keon Johnson,0


In [47]:
first_row_per_player = data.drop_duplicates(subset='Player', keep='first')

In [49]:
first_row_per_player.rename(columns = {'season':'Year'}, inplace = True)

/var/folders/y_/grgdfgkn6ydcldp4b25ckwwr0000gn/T/ipykernel_63171/619004270.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  first_row_per_player.rename(columns = {'season':'Year'}, inplace = True)


In [51]:
draft_player = pd.merge(first_row_per_player, draft, on = ['Player', 'Year'])

In [53]:
draft_player = draft_player[['player_id', 'Overall Pick']]

In [55]:
data_final = pd.merge(data_final, draft_player, on=['player_id'])

In [57]:
data_final

,games_played,player_id,Year,min,fgm,fga,fg3m,fg3a,ftm,fta,oreb,dreb,reb,ast,stl,blk,turnover,pf,pts,fg_pct,fg3_pct,ft_pct,first_name,last_name,position,height_feet,height_inches,weight_pounds,Player,Rookie,Overall Pick
0,13.0,46398676,1960,21.00,3.385,8.846,0.0,0.0,0.692,1.077,0.0,0.0,1.923,2.308,0.0,0.0,0.0,2.692,7.462,0.383,0.000,0.643,Alvin,Attles,NaN,NaN,NaN,NaN,Alvin Attles,0,39
1,13.0,46398674,1960,20.55,2.000,6.846,0.0,0.0,0.615,0.923,0.0,0.0,2.385,0.462,0.0,0.0,0.0,1.615,4.615,0.292,0.000,0.667,Ralph,Davis,NaN,NaN,NaN,NaN,Ralph Davis,0,17
2,12.0,46398666,1960,16.45,2.167,5.250,0.0,0.0,0.833,1.750,0.0,0.0,5.000,0.083,0.0,0.0,0.0,2.333,5.167,0.413,0.000,0.476,Darrall,Imhoff,NaN,NaN,NaN,NaN,Darrall Imhoff,0,3
3,4.0,46398680,1960,10.30,1.500,4.750,0.0,0.0,1.500,2.500,0.0,0.0,3.250,0.000,0.0,0.0,0.0,0.500,4.500,0.316,0.000,0.600,Ron,Johnson,NaN,NaN,NaN,NaN,Ron Johnson,0,12
4,5.0,46398673,1960,8.36,1.000,2.800,0.0,0.0,0.200,0.600,0.0,0.0,1.800,0.400,0.0,0.0,0.0,0.400,2.200,0.357,0.000,0.333,Howie,Jolliff,NaN,NaN,NaN,NaN,Howie Jolliff,0,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18172,31.0,38017729,2021,32.90,7.200,12.500,1.5,3.4,3.400,4.900,NaN,NaN,7.500,2.900,1.2,1.1,2.3,2.300,19.200,0.571,0.438,0.691,NaN,NaN,NaN,NaN,NaN,NaN,David Roddy,1,23
18173,30.0,38017731,2021,32.40,4.600,9.600,2.2,5.6,2.800,3.400,NaN,NaN,6.000,3.000,1.6,1.1,2.9,2.900,14.100,0.477,0.387,0.814,NaN,NaN,NaN,NaN,NaN,NaN,Vince Williams Jr.,1,47
18174,27.0,46406985,1995,33.30,6.900,17.700,2.1,7.3,4.200,5.800,NaN,NaN,6.100,2.600,1.8,0.6,3.0,2.300,20.100,0.389,0.293,0.720,NaN,NaN,NaN,NaN,NaN,NaN,Ron Riley,1,38
18175,27.0,46406985,1995,33.30,6.900,17.700,2.1,7.3,4.200,5.800,NaN,NaN,6.100,2.600,1.8,0.6,3.0,2.300,20.100,0.389,0.293,0.720,NaN,NaN,NaN,NaN,NaN,NaN,Ron Riley,1,38


In [59]:
# concat old and new and make sure rookie column is aligned

In [61]:
#del data['Player']
#del data['Year']

In [63]:
data_final = data_final.sort_values(by='Year', ascending=True)

In [65]:
data_x_list = []
data_y_list = []

In [67]:
del data_final['Player']

In [69]:
data_final.drop(['first_name', 'last_name', 'position', 'height_feet', 'height_inches', 'weight_pounds'], axis = 1, inplace = True)

In [71]:
import numpy as np
import pandas as pd

stats_features = ['games_played', 'min', 'fgm', 'fga', 'fg3m',
       'turnover', 'fg3a', 'ftm', 'fta', 'oreb', 'dreb', 'reb', 'ast', 'stl', 'blk', 'pf', 'pts', 'fg_pct', 'fg3_pct', 'ft_pct'] 
stats_target   = ['games_played', 'pts','fgm', 'fga', 'fg3m', 'ftm', 'fta','reb', 'ast', 'stl', 'blk', 'turnover']   
max_lags = 22              

# 1) Build one row per (Player,Year), with all K stats
df = (
    data_final
      .reset_index(drop=True)                      # ← clear any MultiIndex or old index
      .rename(columns={'player_id':'Player',
                       'season':'Year'})
      [['Player','Year'] + stats_features]         # select only those columns
      .groupby(['Player','Year'], as_index=False) # group on the columns
      .mean()
)



# 2) Collect samples
samples = []   # each is dict(Player,Year,seq_len,X (k×K), y (scalar or K‐vector))
for player, grp in df.groupby('Player'):
    grp = grp.sort_values('Year').reset_index(drop=True)
    feat_arr = grp[stats_features].values   # shape (T, K)
    targ_arr = grp[stats_target].values     # shape (T, len(stats_target))
    years    = grp['Year'].values

    # look‑ahead: for k in 1…T-1, use [0:k] → predict [k]
    for k in range(1, len(grp)):
        samples.append({
            'Player': player,
            'Year':   years[k],
            'seq_len': k,
            'X':       feat_arr[:k],     # (k, K)
            'y':       targ_arr[k]       # scalar if len(stats_target)==1, else (K,)
        })

# 3) Allocate padded arrays
n_samples = len(samples)
K_in      = len(stats_features)
K_out     = len(stats_target)

_, K_in = samples[0]['X'].shape
y0 = samples[0]['y']
# if y0 is a scalar, ndim==0; else ndim==1 and its length is K_out
if np.ndim(y0) == 0:
    K_out = None
else:
    K_out = y0.shape[0]

# X: (n_samples, max_lags, K_in)
data_x = np.full((n_samples, max_lags, K_in), np.nan, dtype=float)

# y: if single‐stat target → (n_samples,), else (n_samples, K_out)
if K_out == 1:
    data_y = np.full((n_samples,), np.nan, dtype=float)
else:
    data_y = np.full((n_samples, K_out), np.nan, dtype=float)

players = [None]*n_samples
years   = [None]*n_samples

print("Example sample X shape:", samples[0]['X'].shape)
print("data_x shape:", data_x.shape)

for i, samp in enumerate(samples):
    L = samp['seq_len']
    data_x[i, :L, :] = samp['X']
    if K_out == 1:
        data_y[i] = samp['y']
    else:
        data_y[i, :] = samp['y']
    players[i] = samp['Player']
    years[i]   = samp['Year']

# 4) (Optional) drop any sample with all‐NaN features
mask = ~np.isnan(data_x).all(axis=(1,2))
data_x  = data_x[mask]
data_y  = data_y[mask]
players = [players[i] for i in np.where(mask)[0]]
years   = [years[i]   for i in np.where(mask)[0]]

Example sample X shape: (1, 20)
data_x shape: (15439, 22, 20)


In [73]:
# check if data creation is accurate

In [75]:
# check why number of rows reduced from 18000 to 14000, im assuming because lot of the rookies did not have high school data

In [77]:
print("X:", data_x.shape, "   # (n_samples, max_lags, K_in)")
print("y:", data_y.shape, "   # (n_samples, ) or (n_samples, K_out)")

X: (15439, 22, 20)    # (n_samples, max_lags, K_in)
y: (15439, 12)    # (n_samples, ) or (n_samples, K_out)


In [79]:
import numpy as np

# 1) Remove any sample whose feature matrix is entirely NaN
mask = ~np.isnan(data_x).all(axis=(1, 2))

data_x = data_x[mask]
data_y = data_y[mask]

# 2) Replace all remaining NaNs with 0
data_x = np.nan_to_num(data_x, nan=0.0)
data_y = np.nan_to_num(data_y, nan=0.0)

# 3) (Optional) If you need to realign metadata lists:
players = [players[i] for i in np.where(mask)[0]]
years   = [years[i]   for i in np.where(mask)[0]]

print("Cleaned data_x shape:", data_x.shape)
print("Cleaned data_y shape:", data_y.shape)

Cleaned data_x shape: (15439, 22, 20)
Cleaned data_y shape: (15439, 12)


In [81]:
# for stat in stats:
#     data_stat = data_train[['player_id', stat, 'season']]
#     data_stat.sort_values(by = 'season', ascending = True, inplace = True)
#     data_stat = data_stat.groupby(['player_id', 'season']).mean().reset_index()
#     data_stat.rename(columns = {'player_id':'Player', 'season':'Year'}, inplace = True)
#     data_stat_final_1 = pd.DataFrame()
#     for i in range(1, 23):
#         locals()["data_stat_"+ str(i)] = data_stat.groupby('Player').nth(i-1)
#         locals()["data_stat_" + str(i)].drop(columns = {'Year'}, inplace = True)
#         locals()["data_stat_"+ str(i)].rename(columns = {stat:stat + '_'+str(i)}, inplace = True)
#     from functools import reduce
#     data_frames = [data_stat_1, data_stat_2, data_stat_3, data_stat_4, data_stat_5, data_stat_6, data_stat_7, data_stat_8, data_stat_9, data_stat_10, data_stat_11, data_stat_12,data_stat_13, data_stat_14, data_stat_15, data_stat_16, data_stat_17, data_stat_18,data_stat_19, data_stat_20, data_stat_21]
#     data_stat_final_1 = reduce(lambda  left,right: pd.merge(left,right,on=['Player'],
#                                                 how='outer'), data_frames)
#     data_stat_final_1 = data_stat_final_1[data_stat_final_1[stat + '_2'].notnull()]
#     data_stat_y_1 = data_stat.groupby('Player').last()
#     data_stat_y_1.drop(columns = {'Year'}, inplace = True)
#     data_stat_y_1  = pd.merge(data_stat_y_1, data_stat_2, on = ['Player'])
#     data_stat_y_1.drop(columns = {stat + '_2'}, inplace = True)
#     # now insert Season as the very first column of data_stat_final_1
#     data_stat_final_1.insert(0, 'Season', data_stat_y_1['Season'].values)
#     last_non_nan_indices = np.where(~np.isnan(data_stat_final_1), np.arange(data_stat_final_1.shape[1])[None, :], -1).max(axis=1)
#     for i in range(0, len(data_stat_final_1)):
#         idx = last_non_nan_indices[i]
#         data_stat_final_1.iloc[i, idx] = np.nan
#     data_stat_final = pd.DataFrame()
#     data_stat_final = pd.concat([data_stat_final, data_stat_final_1], ignore_index=True)
#     data_stat_y = pd.DataFrame()
#     data_stat_y = pd.concat([data_stat_y, data_stat_y_1], ignore_index=True)
#     for i in range(2, 21):
        
#         locals()["data_stat_y_"+str(i)] = data_stat.groupby('Player').nth(-i)
#         locals()["data_stat_y_"+str(i)].drop(columns = {'Year'}, inplace = True)
    
#         locals()["data_stat_y_"+str(i)]  = pd.merge(locals()["data_stat_y_"+str(i)], locals()["data_stat_"+str(i+1)], on = ['Player'])
    
#         locals()["data_stat_y_"+str(i)].drop(columns = {stat+ '_'+str(i+1)}, inplace = True)
        
#         # Find the indices of the last non-NaN values in each row
#         last_non_nan_indices = np.where(~np.isnan(locals()["data_stat_final_"+str(i-1)]), np.arange(locals()["data_stat_final_"+str(i-1)].shape[1])[None, :], -1).max(axis=1)
#         locals()["data_stat_final_"+str(i)] = copy.deepcopy(locals()["data_stat_final_"+str(i-1)])
    
#         for j in range(0, len(locals()["data_stat_final_"+str(i)])):
#             idx = last_non_nan_indices[j]
#             locals()["data_stat_final_"+str(i)].iloc[j, idx] = np.nan
    
#         locals()["data_stat_final_"+str(i)] = locals()["data_stat_final_"+str(i)][locals()["data_stat_final_"+str(i)][stat + '_1'].notnull()]
#         data_stat_final = pd.concat([data_stat_final, locals()["data_stat_final_"+str(i)]], ignore_index = True)
#         data_stat_y = pd.concat([data_stat_y, locals()["data_stat_y_"+str(i)]], ignore_index = True)
#     data_stat_final.fillna(0, inplace = True)

#     del data_stat_final['Player']
#     del data_stat_y['Player']
    
#     data_x_list.append(data_stat_final)
#     if stat in stats_prediction:
#         data_y_list.append(data_stat_y)

In [83]:
#data_x_list

In [85]:
# data_x = np.stack(data_x_list, axis=2)
# data_y = np.stack(data_y_list, axis=1)
# data_y = np.squeeze(data_y, axis=-1)

In [87]:
#data_x.shape

In [89]:
#data_y.shape

In [91]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Reshape to 2D to apply scaler
n_samples, n_timestamps, n_features = data_x.shape
data_x_reshaped = data_x.reshape(-1, n_features)

# Fit scaler and transform
scaler_x = MinMaxScaler()
data_x_scaled = scaler_x.fit_transform(data_x_reshaped)

# Reshape back to 3D
data_x = data_x_scaled.reshape(n_samples, n_timestamps, n_features)

In [93]:
scaler_y = MinMaxScaler()
data_y = scaler_y.fit_transform(data_y)

In [99]:
n_samples, max_lags, K_in = data_x.shape

# 1) ENCODE player IDs as integers 0…P-1
uniq_players, inv_players = np.unique(players, return_inverse=True)
# inv_players[i] is the integer code for players[i]

# 2) NORMALIZE those codes to [0,1] (optional but helps training)
player_feat = inv_players / float(inv_players.max())   # shape (n_samples,)

# 3) NORMALIZE years to [0,1]
years_arr  = np.array(years)
year_feat  = (years_arr - years_arr.min()) / (years_arr.max() - years_arr.min())

# 4) EXPAND them along the time-axis so they match data_x
#    shape → (n_samples, max_lags, 1)
player_feat_seq = np.repeat(player_feat[:,None,None], max_lags, axis=1)
year_feat_seq   = np.repeat(year_feat[:,None,None],   max_lags, axis=1)

# 5) CONCATENATE onto data_x as two new channels
#    new shape = (n_samples, max_lags, K_in + 2)
data_x = np.concatenate([data_x, year_feat_seq, player_feat_seq], axis=2)

In [101]:
# ── build a clean meta‐table with exactly one row per Player–Year ──
meta_df = (
    data_final
      .rename(columns={'player_id':'Player','season':'Year'})
      # keep only the columns you need
      [['Player','Year','Rookie','Overall Pick']]
      # drop any duplicate Player–Year so that the index will be unique
      .drop_duplicates(subset=['Player','Year'])
      # now set a unique MultiIndex
      .set_index(['Player','Year'])
)

# ── now align it to your samples ──
import pandas as pd

# build the MultiIndex matching your samples
idx = pd.MultiIndex.from_arrays([players, years], names=['Player','Year'])

# reindex safely (index is now unique)
meta = meta_df.reindex(idx)

# extract arrays
rookie_arr = meta['Rookie'].astype(float).values       # (n_samples,)
pick_arr   = meta['Overall Pick'].astype(float).values # (n_samples,)

# optional: normalize pick into [0,1]
pick_arr = (pick_arr - pick_arr.min()) / (pick_arr.max() - pick_arr.min())

# expand to time‐axis
n_samples, max_lags, _ = data_x.shape
rookie_seq = np.repeat(rookie_arr[:,None,None], max_lags, axis=1)
pick_seq   = np.repeat(pick_arr[:,  None,None], max_lags, axis=1)

# concatenate onto data_x
# (remember you already did: data_x = np.concatenate([..., year_feat_seq, player_feat_seq], axis=2))
data_x = np.concatenate([data_x, rookie_seq, pick_seq], axis=2)

print("Final data_x shape:", data_x.shape)

Final data_x shape: (15439, 22, 26)


In [103]:
# 1) turn years into a NumPy array
years_arr = np.array(years)

# 2) define test mask for seasons 2020–2023
test_years = [2023]
test_mask  = np.isin(years_arr, test_years)

# 3) the rest are train+val candidates
all_idx         = np.arange(len(years_arr))
train_val_idx   = all_idx[~test_mask]
train_val_years = years_arr[~test_mask]

# 4) sort those by ascending year
sorted_tv_idx = train_val_idx[np.argsort(train_val_years)]

# 5) split 80% train / 20% validation of train_val
n_tv    = len(sorted_tv_idx)
n_val   = int(np.floor(0.25 * n_tv))
train_idx = sorted_tv_idx[:-n_val] if n_val > 0 else sorted_tv_idx
val_idx   = sorted_tv_idx[-n_val:] if n_val > 0 else np.array([], dtype=int)

# 6) test indices
test_idx = all_idx[test_mask]

# 7) slice out your sets
train_x = data_x[train_idx]
train_y = data_y[train_idx]
players_train = [players[i] for i in train_idx]
years_train   = years_arr[train_idx]

valid_x = data_x[val_idx]
valid_y = data_y[val_idx]
players_val = [players[i] for i in val_idx]
years_val   = years_arr[val_idx]

test_x = data_x[test_idx]
test_y = data_y[test_idx]
players_test = [players[i] for i in test_idx]
years_test   = years_arr[test_idx]

In [105]:
train_x.shape

(11337, 22, 26)

In [107]:
train_y.shape

(11337, 12)

# Training LSTM Model

In [110]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout, ConvLSTM2D, Flatten
from tensorflow.keras.layers import Masking
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score

In [127]:
model = Sequential()
model.add(Masking(mask_value=0., input_shape = (22, 26)))
model.add(LSTM(128, activation = 'relu', return_sequences = True))
model.add(LSTM(64, activation = 'relu'))
model.add(Dense(32))
model.add(Dense(12))
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [129]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ masking_3 (Masking)             │ (None, 22, 26)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_6 (LSTM)                   │ (None, 22, 128)        │        79,360 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_7 (LSTM)                   │ (None, 64)             │        49,408 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 12)             │           396 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 131,244 (512.67 KB)

 Trainable params: 131,244 (512.67 KB)

 Non-trainable params: 0 (0.00 B)

In [131]:
model.fit(train_x, train_y, validation_data = (valid_x, valid_y), verbose = 2, epochs = 5)

Epoch 1/5
355/355 - 5s - 13ms/step - loss: 0.0187 - val_loss: 0.0141
Epoch 2/5
355/355 - 4s - 12ms/step - loss: 0.0126 - val_loss: 0.0123
Epoch 3/5
355/355 - 4s - 12ms/step - loss: 0.0115 - val_loss: 0.0121
Epoch 4/5
355/355 - 4s - 12ms/step - loss: 0.0111 - val_loss: 0.0122
Epoch 5/5
355/355 - 4s - 12ms/step - loss: 0.0107 - val_loss: 0.0113


In [132]:
train_predict = model.predict(train_x)

355/355 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [136]:
from sklearn.metrics import r2_score

# Compute and print R² for each feature
for i, stat in enumerate(stats_target):
    score = r2_score(train_y[:, i], train_predict[:, i])
    print(f"{stat}: R² = {score:.4f}")

games_played: R² = 0.2488
pts: R² = 0.7263
fgm: R² = 0.6969
fga: R² = 0.7136
fg3m: R² = 0.6016
ftm: R² = 0.6946
fta: R² = 0.6942
reb: R² = 0.6777
ast: R² = 0.7028
stl: R² = 0.6010
blk: R² = 0.6732
turnover: R² = 0.6925


In [138]:
train_y_original = scaler_y.inverse_transform(train_y)
train_predict_original = scaler_y.inverse_transform(train_predict)

In [142]:
from sklearn.metrics import mean_absolute_error

# Compute MAE per output column
mae_list = [
    mean_absolute_error(train_y_original[:, i], train_predict_original[:, i])
    for i in range(train_y_original.shape[1])
]

# Print labeled MAEs
for stat, mae in zip(stats_target, mae_list):
    print(f"{stat}: MAE = {mae:.4f}")

games_played: MAE = 17.1357
pts: MAE = 2.5885
fgm: MAE = 1.0535
fga: MAE = 2.0754
fg3m: MAE = 0.2164
ftm: MAE = 0.6396
fta: MAE = 0.8187
reb: MAE = 1.1440
ast: MAE = 0.7375
stl: MAE = 0.2310
blk: MAE = 0.2022
turnover: MAE = 0.3724


In [144]:
# Test data
test_predict = model.predict(test_x)

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 


In [148]:
from sklearn.metrics import r2_score

# Compute and print R² for each feature
for i, stat in enumerate(stats_target):
    score = r2_score(test_y[:, i], test_predict[:, i])
    print(f"{stat}: R² = {score:.4f}")

games_played: R² = -0.6649
pts: R² = 0.8255
fgm: R² = 0.8133
fga: R² = 0.8155
fg3m: R² = 0.6455
ftm: R² = 0.7826
fta: R² = 0.7830
reb: R² = 0.6135
ast: R² = 0.5299
stl: R² = 0.4338
blk: R² = 0.4909
turnover: R² = 0.6789


In [150]:
# see if there is another way to train the LSTM network: done

# Rolling window LSTM

In [153]:
# more epochs and train 

In [155]:
from sklearn.metrics import r2_score
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Masking, LSTM, Dense

# helper to build your model
def build_model(input_shape, output_dim):
    m = Sequential([
        Masking(mask_value=0., input_shape=input_shape),
        LSTM(128, activation='tanh', return_sequences=True),
        LSTM(64, activation='tanh'            ),
        Dense(32, activation='relu'),
        Dense(output_dim)
    ])
    m.compile(loss='mean_squared_error', optimizer='adam')
    return m

# 0) set up
years_arr  = np.array(years)
all_years  = sorted(np.unique(years_arr))
K          = data_y.shape[1]    # number of targets

# 1) build the model a single time
model = build_model(
    input_shape=(data_x.shape[1], data_x.shape[2]),
    output_dim=K
)

# 2) expanding-window training
for year in all_years[1:]:
    train_mask = years_arr <  year
    test_mask  = years_arr == year

    X_tr, y_tr = data_x[train_mask], data_y[train_mask]
    X_te, y_te = data_x[test_mask],  data_y[test_mask]

    print(f"\n→ Training on years < {year}: {X_tr.shape[0]} samples")
    # this will continue training on the cumulative data
    model.fit(
        X_tr, y_tr,
        epochs=20,
        shuffle=False,   # preserve temporal order
        verbose=2
    )

    # evaluate on the hold-out year
    y_pred = model.predict(X_te)
    for i, stat in enumerate(stats_target):
        print(f"  {stat} R² = {r2_score(y_te[:,i], y_pred[:,i]):.4f}")


→ Training on years < 1962: 14 samples
Epoch 1/20


/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 - 1s - 896ms/step - loss: 0.0695
Epoch 2/20
1/1 - 0s - 17ms/step - loss: 0.0525
Epoch 3/20
1/1 - 0s - 17ms/step - loss: 0.0446
Epoch 4/20
1/1 - 0s - 16ms/step - loss: 0.0423
Epoch 5/20
1/1 - 0s - 16ms/step - loss: 0.0412
Epoch 6/20
1/1 - 0s - 14ms/step - loss: 0.0396
Epoch 7/20
1/1 - 0s - 15ms/step - loss: 0.0379
Epoch 8/20
1/1 - 0s - 15ms/step - loss: 0.0368
Epoch 9/20
1/1 - 0s - 15ms/step - loss: 0.0359
Epoch 10/20
1/1 - 0s - 14ms/step - loss: 0.0351
Epoch 11/20
1/1 - 0s - 14ms/step - loss: 0.0345
Epoch 12/20
1/1 - 0s - 14ms/step - loss: 0.0339
Epoch 13/20
1/1 - 0s - 16ms/step - loss: 0.0333
Epoch 14/20
1/1 - 0s - 14ms/step - loss: 0.0330
Epoch 15/20
1/1 - 0s - 14ms/step - loss: 0.0328
Epoch 16/20
1/1 - 0s - 14ms/step - loss: 0.0324
Epoch 17/20
1/1 - 0s - 14ms/step - loss: 0.0320
Epoch 18/20
1/1 - 0s - 15ms/step - loss: 0.0316
Epoch 19/20
1/1 - 0s - 14ms/step - loss: 0.0313
Epoch 20/20
1/1 - 0s - 15ms/step - loss: 0.0309
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
  games_played R² = -

# Analysis on LSTM

# Training Transformer 

In [156]:
from sklearn.metrics import r2_score
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import (
    Masking, Dense, Dropout, LayerNormalization,
    MultiHeadAttention, GlobalAveragePooling1D
)

# 0) setup
years_arr = np.array(years)
all_years = sorted(np.unique(years_arr))
K         = data_y.shape[1]          # number of targets

# 1) define a Transformer-block layer
def transformer_block(x, embed_dim=64, num_heads=4, ff_dim=64, rate=0.1):
    # multi-head self attention
    attn = MultiHeadAttention(
        num_heads=num_heads,
        key_dim=embed_dim//num_heads,
        dropout=rate
    )(x, x)
    attn = Dropout(rate)(attn)
    x    = LayerNormalization(epsilon=1e-6)(x + attn)

    # feed-forward
    ffn  = Dense(ff_dim, activation="relu")(x)
    ffn  = Dense(embed_dim)(ffn)
    ffn  = Dropout(rate)(ffn)
    return LayerNormalization(epsilon=1e-6)(x + ffn)

# 2) build your Transformer model once
def build_transformer(input_shape, output_dim,
                      embed_dim=64, num_heads=4, ff_dim=64, num_layers=2, rate=0.1):
    inp = Input(shape=input_shape)                 # (timesteps, features)
    x   = Masking(mask_value=0.0)(inp)
    # project features → model dimension
    x   = Dense(embed_dim)(x)

    # stack N Transformer blocks
    for _ in range(num_layers):
        x = transformer_block(x, embed_dim, num_heads, ff_dim, rate)

    # pool and output
    x   = GlobalAveragePooling1D()(x)
    x   = Dropout(rate)(x)
    x   = Dense(ff_dim, activation="relu")(x)
    out = Dense(output_dim)(x)

    model = Model(inp, out)
    model.compile(optimizer="adam", loss="mean_squared_error")
    return model

model_trans = build_transformer(
    input_shape=(data_x.shape[1], data_x.shape[2]),
    output_dim=K,
    embed_dim=64,
    num_heads=4,
    ff_dim=64,
    num_layers=2,
    rate=0.1
)

# 3) expanding-window training and evaluation
for year in all_years[1:]:
    train_mask = years_arr <  year
    test_mask  = years_arr == year

    X_tr, y_tr = data_x[train_mask], data_y[train_mask]
    X_te, y_te = data_x[test_mask],  data_y[test_mask]

    print(f"\n→ Transformer: train on <{year} ({X_tr.shape[0]} samples), test on {year} ({X_te.shape[0]})")
    model_trans.fit(
        X_tr, y_tr,
        epochs=20,
        shuffle=False,
        verbose=2
    )

    y_pred = model_trans.predict(X_te)
    for i, stat in enumerate(stats_target):
        print(f"  {stat} R² = {r2_score(y_te[:,i], y_pred[:,i]):.4f}")


→ Transformer: train on <1962 (14 samples), test on 1962 (23)
Epoch 1/20
1/1 - 1s - 1s/step - loss: 0.8429
Epoch 2/20
1/1 - 0s - 12ms/step - loss: 0.3642
Epoch 3/20
1/1 - 0s - 12ms/step - loss: 0.1699
Epoch 4/20
1/1 - 0s - 11ms/step - loss: 0.1772
Epoch 5/20
1/1 - 0s - 11ms/step - loss: 0.1953
Epoch 6/20
1/1 - 0s - 12ms/step - loss: 0.1457
Epoch 7/20
1/1 - 0s - 11ms/step - loss: 0.1345
Epoch 8/20
1/1 - 0s - 11ms/step - loss: 0.1394
Epoch 9/20
1/1 - 0s - 11ms/step - loss: 0.0896
Epoch 10/20
1/1 - 0s - 11ms/step - loss: 0.1206
Epoch 11/20
1/1 - 0s - 12ms/step - loss: 0.1115
Epoch 12/20
1/1 - 0s - 11ms/step - loss: 0.0956
Epoch 13/20
1/1 - 0s - 12ms/step - loss: 0.1136
Epoch 14/20
1/1 - 0s - 12ms/step - loss: 0.0914
Epoch 15/20
1/1 - 0s - 12ms/step - loss: 0.0957
Epoch 16/20
1/1 - 0s - 12ms/step - loss: 0.0964
Epoch 17/20
1/1 - 0s - 12ms/step - loss: 0.0834
Epoch 18/20
1/1 - 0s - 11ms/step - loss: 0.0737
Epoch 19/20
1/1 - 0s - 12ms/step - loss: 0.1125
Epoch 20/20
1/1 - 0s - 11ms/step - l

# Analysis on Transformer